# Dependencies

In [0]:

"""A deep MNIST classifier using convolutional layers.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/pros
"""
# Disable linter warnings to maintain consistency with tutorial.
# pylint: disable=invalid-name
# pylint: disable=g-bad-import-order

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
from numpy import linalg as LA


# Model

In [0]:
def conv2d(x, W):
  """conv2d returns a 2d convolution layer with full stride."""
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def max_pool_2x2(x):
  """max_pool_2x2 downsamples a feature map by 2X."""
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def weight_variable(shape,name=''):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial,name=name)


def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

class CNN():
  def create_dict(self):
    pass
  def __init__(self):
    self.layers = ['conv1/W_conv1','conv2/W_conv2','fc1/W_fc1','fc2/W_fc2']
    self.x = tf.placeholder(tf.float32, [None, 784],name='x')
    # Define loss and optimizer                                                   
    self.y_ = tf.placeholder(tf.float32, [None, 10],name='y')
    with tf.name_scope('reshape'):
      self.x_image = tf.reshape(self.x, [-1, 28, 28, 1],name='x_image')
    with tf.name_scope('conv1'):
      self.W_conv1 = weight_variable([5, 5, 1, 20],'W_conv1')
      b_conv1 = bias_variable([20])
      h_conv1 = tf.nn.relu(conv2d(self.x_image, self.W_conv1) + b_conv1)
    with tf.name_scope('pool1'):
      h_pool1 = max_pool_2x2(h_conv1)
    with tf.name_scope('conv2'):
      self.W_conv2 = weight_variable([5, 5, 20, 50],'W_conv2')
      b_conv2 = bias_variable([50])
      h_conv2 = tf.nn.relu(conv2d(h_pool1, self.W_conv2) + b_conv2)
    with tf.name_scope('pool2'):
      h_pool2 = max_pool_2x2(h_conv2)
    with tf.name_scope('fc1'):
      self.W_fc1 = weight_variable([4 * 4 * 50, 500],'W_fc1')
      b_fc1 = bias_variable([500])
      h_pool2_flat = tf.reshape(h_pool2, [-1, 4*4*50])
      h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, self.W_fc1) + b_fc1)
    with tf.name_scope('dropout'):
      self.keep_prob = tf.placeholder(tf.float32)
      h_fc1_drop = tf.nn.dropout(h_fc1, self.keep_prob)
    with tf.name_scope('fc2'):
      self.W_fc2 = weight_variable([500, 10],'W_fc2')
      b_fc2 = bias_variable([10])
    self.logits = tf.add(tf.matmul(h_fc1_drop, self.W_fc2),b_fc2,name='logits')
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.y_,
                                                                logits=self.logits)
    self.cross_entropy = tf.reduce_mean(cross_entropy,name='entropy_loss')
def create_model():
  return CNN()

# Prune

In [0]:
class PruneConfiguration():
  P1 = 80
  P2 = 92
  P3 = 99.1
  P4 = 93  
  @staticmethod
  def display():
    print("P1 is %f"% PruneConfiguration.P1)
    print("P2 is %f"% PruneConfiguration.P2)
    print("P3 is %f"% PruneConfiguration.P3)
    print("P4 is %f"% PruneConfiguration.P4)
configuration = PruneConfiguration()

target_w = ['conv1/W_conv1','conv2/W_conv2','fc1/W_fc1','fc2/W_fc2']
prune_percent = {'conv1/W_conv1':configuration.P1,'conv2/W_conv2':configuration.P2,'fc1/W_fc1':configuration.P3,'fc2/W_fc2':configuration.P4}

def get_configuration():
  return configuration

def projection(weight_arr,percent = 10):
    pcen = np.percentile(abs(weight_arr),percent)
    print ("percentile " + str(pcen))
    under_threshold = abs(weight_arr)< pcen
    weight_arr[under_threshold] = 0
    return weight_arr


def prune_weight(weight_arr,weight_name):
  # to work with admm, we calculate percentile based on all elements instead of nonzero elements.
  percent = prune_percent[weight_name]
  pcen = np.percentile(abs(weight_arr),percent)
  print ("percentile " + str(pcen))
  under_threshold = abs(weight_arr)< pcen
  weight_arr[under_threshold] = 0
  above_threshold = abs(weight_arr)>= pcen
  return [above_threshold,weight_arr]
def apply_prune(dense_w,sess):
    # returns dictionary of non_zero_values' indices                                                                                                                   
  dict_nzidx = {}
  for target_name in target_w:
    print ("at weight "+target_name)
    weight_arr = sess.run(dense_w[target_name])
    print ("before pruning #non zero parameters " + str(np.sum(weight_arr!=0)))
    before = np.sum(weight_arr!=0)
    mask,weight_arr_pruned = prune_weight(weight_arr,target_name)
    after = np.sum(weight_arr_pruned!=0)
    print ("pruned "+ str(before-after))
    
    print ("after prunning #non zero parameters " + str(np.sum(weight_arr_pruned!=0)))
    sess.run(dense_w[target_name].assign(weight_arr_pruned))
    dict_nzidx[target_name] = mask
  return dict_nzidx
def apply_prune_on_grads(grads_and_vars,dict_nzidx):

  for key, nzidx in dict_nzidx.items():    
    count = 0
    for grad, var in grads_and_vars:
      if var.name == key+":0":
        nzidx_obj = tf.cast(tf.constant(nzidx), tf.float32)
        grads_and_vars[count] = (tf.multiply(nzidx_obj, grad), var)
      count += 1
  return grads_and_vars


# AdmmSolver

In [0]:
class AdmmSolver():
  def __init__(self,model):    
    A=self.A = tf.placeholder(tf.float32, shape = [5, 5, 1, 20])
    B=self.B = tf.placeholder(tf.float32, shape = [5, 5, 1, 20])
    C=self.C = tf.placeholder(tf.float32, shape = [5, 5, 20, 50])
    D=self.D = tf.placeholder(tf.float32, shape = [5, 5, 20, 50])
    E=self.E = tf.placeholder(tf.float32, shape = [4 * 4 * 50, 500])
    F=self.F = tf.placeholder(tf.float32, shape = [4 * 4 * 50, 500])
    G=self.G = tf.placeholder(tf.float32, shape = [500, 10])
    H=self.H = tf.placeholder(tf.float32, shape = [500, 10])

    W_conv1 = model.W_conv1
    W_conv2 = model.W_conv2
    W_fc1 = model.W_fc1
    W_fc2 = model.W_fc2
    cross_entropy = model.cross_entropy
    with tf.name_scope('adam_optimizer'):
        train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy+0.00005*(tf.nn.l2_loss(W_conv1)+tf.nn.l2_loss(W_conv2)+tf.nn.l2_loss(W_fc1)+tf.nn.l2_loss(W_fc2)))
        train_step1 = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy+0.00005*(tf.nn.l2_loss(W_conv1)+tf.nn.l2_loss(W_conv2)+tf.nn.l2_loss(W_fc1)+tf.nn.l2_loss(W_fc2))+0.0001*(tf.nn.l2_loss(W_conv1-A+B)+ tf.nn.l2_loss(W_conv2-C+D)+tf.nn.l2_loss(W_fc1-E+F)+tf.nn.l2_loss(W_fc2-G+H)))
    self.train_step = train_step
    self.train_step1 = train_step1

def create_admm_solver(model):
  return AdmmSolver(model)

# Main

In [0]:
FLAGS = None
# pruning ratio


prune_configuration = get_configuration()
dense_w = {}
P1 = prune_configuration.P1
P2 = prune_configuration.P2
P3 = prune_configuration.P3
P4 = prune_configuration.P4

prune_configuration.display()


def main(_):
  # Import data
  mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)

  model = create_model()
  x = model.x
  y_ = model.y_
  cross_entropy = model.cross_entropy
  layers = model.layers
  logits = model.logits
  solver = create_admm_solver(model)
  keep_prob = model.keep_prob
  train_step = solver.train_step
  train_step1 = solver.train_step1
  
  W_conv1 = model.W_conv1
  W_conv2 = model.W_conv2
  W_fc1 = model.W_fc1
  W_fc2 = model.W_fc2
  
  A = solver.A
  B = solver.B
  C = solver.C
  D = solver.D
  E = solver.E
  F = solver.F
  G = solver.G
  H = solver.H

  my_trainer = tf.train.AdamOptimizer(1e-3)
  grads = my_trainer.compute_gradients(cross_entropy)
    
  with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
  accuracy = tf.reduce_mean(correct_prediction)

  graph_location = tempfile.mkdtemp()
  print('Saving graph to: %s' % graph_location)
  train_writer = tf.summary.FileWriter(graph_location)
  train_writer.add_graph(tf.get_default_graph())

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

    Z1 = sess.run(W_conv1)
    Z1 = projection(Z1, percent=P1)

    U1 = np.zeros_like(Z1)

    Z2 = sess.run(W_conv2)
    Z2 = projection(Z2, percent=P2)

    U2 = np.zeros_like(Z2)

    Z3 = sess.run(W_fc1)
    Z3 = projection(Z3, percent=P3)

    U3 = np.zeros_like(Z3)

    Z4 = sess.run(W_fc2)
    Z4 = projection(Z4, percent=P4)

    U4 = np.zeros_like(Z4)

    for j in range(30):
        for i in range(2000):
            batch = mnist.train.next_batch(50)
            if i % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                    x: batch[0], y_: batch[1], keep_prob: 1.0})
                print('step %d, training accuracy %g' % (i, train_accuracy))
            train_step1.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0, A:Z1, B:U1, C:Z2, D:U2, E:Z3, F:U3, G:Z4, H:U4})
        Z1 = sess.run(W_conv1) + U1
        Z1 = projection(Z1, percent=P1)

        U1 = U1 + sess.run(W_conv1) - Z1

        Z2 = sess.run(W_conv2) + U2
        Z2 = projection(Z2, percent=P2)

        U2 = U2 + sess.run(W_conv2) - Z2

        Z3 = sess.run(W_fc1) + U3
        Z3 = projection(Z3, percent=P3)

        U3 = U3 + sess.run(W_fc1) - Z3

        Z4 = sess.run(W_fc2) + U4
        Z4 = projection(Z4, percent=P4)

        U4 = U4 + sess.run(W_fc2) - Z4

        print('test accuracy %g' % accuracy.eval(feed_dict={
            x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        print(LA.norm(sess.run(W_conv1) - Z1))
        print(LA.norm(sess.run(W_conv2) - Z2))
        print(LA.norm(sess.run(W_fc1) - Z3))
        print(LA.norm(sess.run(W_fc2) - Z4))

    dense_w['conv1/W_conv1'] = W_conv1
    dense_w['conv2/W_conv2'] = W_conv2
    dense_w['fc1/W_fc1'] = W_fc1
    dense_w['fc2/W_fc2'] = W_fc2
    
    #applying pruning function
    dict_nzidx = apply_prune(dense_w,sess)
    print ("checking space dictionary")
    print (dict_nzidx.keys())
    # applying pruning on grads
    grads = apply_prune_on_grads(grads,dict_nzidx)
    apply_gradient_op = my_trainer.apply_gradients(grads)
    for var in tf.global_variables():
                if tf.is_variable_initialized(var).eval() == False:
                    sess.run(tf.variables_initializer([var]))
    print ("start retraining after pruning")
    for i in range(5000):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))

      apply_gradient_op.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print('test accuracy %g' % accuracy.eval(feed_dict={
          x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
    print(np.sum(sess.run(W_conv1)!=0))
    print(np.sum(sess.run(W_conv2) != 0))
    print(np.sum(sess.run(W_fc1) != 0))
    print(np.sum(sess.run(W_fc2) != 0))
    # do the saving.
    saver = tf.train.Saver()
    saver.save(sess,"./lenet_5_pruned_model.ckpt")
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('--data_dir', type=str,
                      default='/tmp/tensorflow/mnist/input_data',
                      help='Directory for storing input data')
  FLAGS, unparsed = parser.parse_known_args()
tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

P1 is 80.000000
P2 is 92.000000
P3 is 99.100000
P4 is 93.000000
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extra

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
